# 1. Instalar e Importar Librerías Necesarias

En esta sección instalaremos e importaremos todas las librerías necesarias para el procesamiento de PDFs, vectorización, tokenización y uso de modelos ligeros multilenguaje compatibles con Ollama.

In [ ]:
# Instalar librerías necesarias
!pip install PyPDF2 sentence-transformers langchain faiss-cpu pdf2image pytesseract pillow ollama

# Importar librerías
import os
import subprocess
from pdf2image import convert_from_path
import pytesseract
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import pickle

# 2. Configuración y seteo de binarios

En esta sección construiremos todas las configuraciones necesarias, path a binario y demás

In [ ]:
# -------------------------------
# Configuración
# -------------------------------
pdf_folder = "C:/Users/Usuario/Desktop/cursos_repos/rag-finetunning/pdfs"
BASE_DIR = "C:/Users/Usuario/Desktop/cursos_repos/rag-finetunning"
INDEX_PATH = os.path.join(BASE_DIR, "faiss_index.bin")
CHUNKS_PATH = os.path.join(BASE_DIR, "chunks.pkl")
EMBEDDINGS_PATH = os.path.join(BASE_DIR, "embeddings.npy")

#tesseract
tesseract_cmd = os.path.join(os.getcwd(), "tesseract", "tesseract.exe")
pytesseract.pytesseract.tesseract_cmd = tesseract_cmd

#poppler
poppler_path = os.path.join(os.getcwd(), "poppler", "bin")

# Verificar que los binarios existen
required_bins = ["pdfinfo.exe", "pdftoppm.exe"]
for b in required_bins:
    if not os.path.exists(os.path.join(poppler_path, b)):
        raise FileNotFoundError(f"No se encontró {b} en {poppler_path}")

# 3. Cargar modelo multilenguaje

En esta sección se carga un modelo para realizar los embeddings. Debe ser el mismo con el cual se lee después

In [ ]:
# Cargar modelo multilenguaje ligero
model_name = "sentence-transformers/distiluse-base-multilingual-cased-v2"
embedder = SentenceTransformer(model_name)

# 3. Función OCR de imágenes dentro del pdf

En esta sección se revisan si hay imagenes en pdf para extraer texto

In [ ]:
# -------------------------------
# Función OCR para PDFs de imágenes
# -------------------------------
def extract_text_from_image_pdf(pdf_file, psm=6):
    pages = convert_from_path(pdf_file, poppler_path=poppler_path)
    all_chunks = []
    for i, page in enumerate(pages):
        text = pytesseract.image_to_string(page, lang='eng+spa', config=f'--psm {psm}').strip()
        if text:
            # Dividir cada página en chunks más pequeños
            words = text.split()
            page_chunks = [' '.join(words[j:j+200]) for j in range(0, len(words), 200)]
            all_chunks.extend(page_chunks)
    return all_chunks

# 4. Check de páginas detectadas

In [ ]:
pdf_file = os.path.join(pdf_folder, "MS-15B1_v1.0_English.pdf")
pages = convert_from_path(pdf_file, poppler_path=poppler_path)
print("Páginas detectadas:", len(pages))

# 5. Procesamos todos los pdfs

En esta sección se recolentan los pdfs y se procesan

In [ ]:
# -------------------------------
# Procesar todos los PDFs
# -------------------------------
pdf_files = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder) if f.lower().endswith('.pdf')]

all_chunks = []
for pdf_file in pdf_files:
    chunks = extract_text_from_image_pdf(pdf_file)
    print(f"{os.path.basename(pdf_file)} -> {len(chunks)} chunks")
    all_chunks.extend(chunks)

print(f"Total de chunks generados: {len(all_chunks)}")

# 6. Generación de embeddings

En esta sección se generan los embeddings

In [ ]:
# -------------------------------
# Generar embeddings
# -------------------------------
if len(all_chunks) == 0:
    raise ValueError("No se generaron chunks. Revisa los PDFs y el OCR.")

embeddings = embedder.encode(all_chunks, show_progress_bar=True, convert_to_numpy=True)
print(f"Se generaron {embeddings.shape[0]} vectores.")


# 7. Creamos el indice FAISS

En esta sección se crea el indice FAISS (basicamente es para manipular mejor los vectores, obtenerlos eficientemente bla bla)

In [ ]:
# -------------------------------
# Crear índice FAISS
# -------------------------------
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)
print(f"Índice FAISS creado con {index.ntotal} vectores.")

# 8. Guardamos indices y datos

En esta sección se guardan los indices

In [ ]:
# -------------------------------
# Guardar índice y datos
# -------------------------------
faiss.write_index(index, INDEX_PATH)
np.save(EMBEDDINGS_PATH, embeddings)
with open(CHUNKS_PATH, "wb") as f:
    pickle.dump(all_chunks, f)
print("FAISS index, embeddings y chunks guardados correctamente.")

# 9. Pulling de modelo ollama

En esta sección cargamos el modelo de ollama

In [ ]:
# -------------------------------
# Función para usar ollama
# -------------------------------
def ollama_pull(model_name="phi"):
    result = subprocess.run(
        ["ollama", "pull", model_name],
        capture_output=True,
        text=True,
        encoding="utf-8",   
        errors="replace"    
    )
    print(result.stdout)

ollama_pull("phi")

# 10. Revisamos cuantos chunks se crearon

En esta sección revisamos cuantos chunks se crearon

In [75]:
# -------------------------------
# Verificación final
# -------------------------------
with open(CHUNKS_PATH, "rb") as f:
    loaded_chunks = pickle.load(f)
print(f"Número de chunks cargados: {len(loaded_chunks)}")
print("Ejemplo de chunk:", loaded_chunks[0][:200], "...")

Número de chunks cargados: 71
Ejemplo de chunk: EL | USER GUIDE ...
